In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.regression import *
import numpy as np
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet

import warnings
import holidays

warnings.filterwarnings(action='ignore') 

In [4]:
# path = f'./processed_data/train_merge.csv'
# df = pd.read_csv(path)
# df = df[df['item']=='BC']
# df['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in df['timestamp']]
# df = pd.get_dummies(df, columns=qual_col, prefix=qual_col)
# # df[~df["Date"].isin(kr_holidays)]


# name_lst = df.corr()['price(원/kg)'].sort_values().keys().to_list()
# value_lst = df.corr()['price(원/kg)'].sort_values().values

# print([a for a, b in zip(name_lst, value_lst) if abs(b)>0.15 ])

In [7]:
# 필요 정보
# item_lst : item 리스트 항목
# qual_col : one-hot-encoder 변환이 필요한 항목

item_lst = ['TG', 'CR', 'CB', 'RD', 'BC']
qual_col = ['corporation', 'location']
columns_dict_train = {
    'TG' : list(set(['Date', 'year', 'month', 'week', 'day', 'supply(kg)_y', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation','price(원/kg)'])),
    'CR' : list(set(['Date', 'year', 'month', 'week', 'day', 'location',   'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'CB' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', '무역수지', '수출 금액', '수출 중량', 'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'RD' : list(set(['Date', 'year', 'month', 'week', 'day', 'corporation', 'location', 'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'BC' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', 'corporation', 'corporation', 'corporation', 'location', 'supply(kg)_y','price(원/kg)']))
}

columns_dict_test= {
    'TG' : list(set(['Date', 'year', 'month', 'week', 'day', 'supply(kg)', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation'])),
    'CR' : list(set(['Date', 'year', 'month', 'week', 'day', 'location',   'corporation', 'location', 'supply(kg)'])),
    'CB' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', '무역수지', '수출 금액', '수출 중량', 'corporation', 'location', 'supply(kg)'])),
    'RD' : list(set(['Date', 'year', 'month', 'week', 'day', 'corporation', 'location', 'corporation', 'location', 'supply(kg)'])),
    'BC' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', 'corporation', 'corporation', 'corporation', 'location', 'supply(kg)']))
}

# item 선택
item = item_lst[0]
path = f'./processed_data/{item}_train_merge.csv'

# 한국 2019~2023 공휴일 정보
kr_holidays = holidays.KR(years=[2019,2020,2021,2022,2023])
#kr_holidays = [str(i) for i in kr_holidays]
kr_holidays = {str(i):kr_holidays[i] for i in kr_holidays}

In [17]:
predictions = []

test = pd.read_csv('./processed_data/test_merge.csv')

for i in range(5):
    item = item_lst[i]
    path = f'./processed_data/{item}_train_merge.csv'

    train = pd.read_csv(path)
    test_df = test[test['item']==item]
    # 데이터 전처리 프로세스
    #train = train[~train["Date"].isin(kr_holidays)] # Holyday 제거
    train['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in train['timestamp']]
    test_df['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in test_df['timestamp']]
    
    for l in qual_col:
        le = LabelEncoder()
        train[l]=le.fit_transform(train[l])
        test_df[l]=le.transform(test_df[l]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

    train = pd.get_dummies(train, columns=['day_name'], prefix=['day_name'])
    test_df = pd.get_dummies(test_df, columns=['day_name'], prefix=['day_name'])

    train = train[columns_dict_train[item]]
    train = train.rename(columns={'supply(kg)_y':'supply(kg)'})
    test_df = test_df[columns_dict_test[item]]

    m = Prophet(seasonality_mode='additive')
    m.fit(train)
    
    test_df.rename(columns={'Date':'ds'},inplace=True)
    forecast = m.predict(test_df)
    predictions.extend(forecast)
    # s = setup(train, target = 'price(원/kg)', 
    #     session_id = 41, 
    #     fold = 10, 
    #     normalize = True, 
    #     normalize_method = 'minmax',
    #     train_size=0.8)

    # reg = create_model('rf')
    # #reg = compare_models()
    # tuned_reg = tune_model(reg, optimize = 'RMSE')
    # prediction = predict_model(tuned_reg, data = test_df)
    # predictions.extend(prediction['prediction_label'])

,Description,Value
0,Session id,41
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(15230, 11)"
4,Transformed data shape,"(15230, 11)"
5,Transformed train set shape,"(12184, 11)"
6,Transformed test set shape,"(3046, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1285.2932,4155386.6227,2038.4765,0.4171,3.2759,0.2794
1,1314.9259,4401527.2642,2097.9817,0.4209,3.2535,0.2762
2,1339.6355,4332410.1438,2081.4442,0.4400,3.3160,0.2703
3,1281.1250,4161542.4105,2039.9859,0.4863,3.1551,0.2765
4,1239.5776,3856114.4566,1963.6992,0.5191,3.1613,0.2608
5,1320.5214,4294784.0999,2072.3861,0.4755,3.3061,0.2468
6,1395.0503,4783126.2313,2187.0359,0.4008,3.4163,0.2902
7,1359.2016,4336960.5249,2082.5370,0.4556,3.3774,0.2675
8,1310.7373,4297545.7127,2073.0523,0.4332,3.3879,0.2676


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:36:15
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
result = pd.read_csv('./data/test.csv')
result['answer'] = predictions

del result['timestamp']
del result['item']
del result['corporation']
del result['location']

for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0


In [ ]:
result

,ID,answer
0,TG_A_J_20230304,2656.057617
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,472.812653
3,TG_A_J_20230307,988.383484
4,TG_A_J_20230308,810.296692
...,...,...
1087,RD_F_J_20230327,-27.032959
1088,RD_F_J_20230328,-260.566620
1089,RD_F_J_20230329,-1594.965820
1090,RD_F_J_20230330,-793.025879


In [ ]:
train

,week,day,corporation,supply(kg),location,month,year,price(원/kg)
0,1.0,1.0,A,695871.0,J,1,2019,0.0
1,1.0,2.0,A,695871.0,J,1,2019,0.0
2,1.0,3.0,A,695871.0,J,1,2019,2559.0
3,1.0,4.0,A,695871.0,J,1,2019,2425.0
4,1.0,5.0,A,695871.0,J,1,2019,2097.0
...,...,...,...,...,...,...,...,...
13702,9.0,2.0,E,61472.0,J,3,2023,3144.0
13703,9.0,3.0,E,61472.0,J,3,2023,3045.0
13704,9.0,1.0,E,61472.0,S,3,2023,3816.0
13705,9.0,2.0,E,61472.0,S,3,2023,3321.0


In [6]:
test_df

NameError: name 'test_df' is not defined